In [4]:
import pandas as pd
df = pd.read_csv('data/aerodynamic_data_set_with_labels.csv')
df.head()


,rle,xup,yup,yxxup,xlo,ylo,yxxlo,alpha_te,beta_te,CL,CD,CM
0,0.00784,0.43320,0.062900,-0.424500,0.36180,-0.058900,0.706900,-0.11260,0.16460,0.808246,0.007262,0.121544
1,0.00852,0.38988,0.069190,-0.466950,0.39798,-0.064790,0.777590,-0.12386,0.14814,0.800081,0.018035,0.126753
2,0.00716,0.47652,0.056610,-0.382050,0.32562,-0.053010,0.636210,-0.10134,0.18106,0.779725,0.003612,0.110895
3,0.00750,0.41154,0.066045,-0.488175,0.41607,-0.050065,0.600865,-0.10697,0.13991,0.796090,0.012809,0.118196
4,0.00886,0.49818,0.053465,-0.403275,0.34371,-0.061845,0.742245,-0.12949,0.17283,0.764920,0.007443,0.107714


In [5]:
df.shape

(500, 12)

separating X and y

In [6]:
X = df.drop(columns=['CL','CD','CM'])
X.head()

,rle,xup,yup,yxxup,xlo,ylo,yxxlo,alpha_te,beta_te
0,0.00784,0.43320,0.062900,-0.424500,0.36180,-0.058900,0.706900,-0.11260,0.16460
1,0.00852,0.38988,0.069190,-0.466950,0.39798,-0.064790,0.777590,-0.12386,0.14814
2,0.00716,0.47652,0.056610,-0.382050,0.32562,-0.053010,0.636210,-0.10134,0.18106
3,0.00750,0.41154,0.066045,-0.488175,0.41607,-0.050065,0.600865,-0.10697,0.13991
4,0.00886,0.49818,0.053465,-0.403275,0.34371,-0.061845,0.742245,-0.12949,0.17283


In [5]:
y1 = df['CL'].values
y2 = df['CD'].values
y3 = df['CM'].values

 Split the dataset into train and test data

In [6]:
exclude = X.drop(columns=['beta_te'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train1, y_test1 = train_test_split(exclude, y1, test_size=0.2, random_state=1)
X_train

,rle,xup,yup,yxxup,xlo,ylo,yxxlo,alpha_te
238,0.006746,0.437938,0.065160,-0.342916,0.400807,-0.070588,0.606388,-0.092367
438,0.007059,0.470090,0.070222,-0.399627,0.433312,-0.063456,0.614672,-0.126235
475,0.007293,0.349606,0.055480,-0.414219,0.386956,-0.052596,0.669898,-0.130106
58,0.007542,0.462983,0.058576,-0.416541,0.314314,-0.069576,0.737827,-0.123156
380,0.008557,0.447753,0.072482,-0.438761,0.424832,-0.057842,0.827846,-0.121485
...,...,...,...,...,...,...,...,...
255,0.006496,0.432862,0.066782,-0.461312,0.410134,-0.047718,0.838891,-0.119022
72,0.008414,0.469751,0.066242,-0.340927,0.310922,-0.060557,0.837235,-0.100988
396,0.008717,0.451137,0.053318,-0.396974,0.336078,-0.052780,0.619090,-0.091751
235,0.007426,0.394618,0.071450,-0.470266,0.364627,-0.052918,0.677078,-0.126147


## CL

Building and training the model

In [7]:
from sklearn.neighbors import KNeighborsRegressor

KNN = KNeighborsRegressor(n_neighbors=10)
KNN.fit(X_train, y_train1)

KNeighborsRegressor(n_neighbors=10)

Check Accuracy

In [8]:
KNN.score(X_test, y_test1)

0.21636245703069523

Increasing the model

Hypertuning model parameters using GridSearchCV

In [10]:
from sklearn.model_selection import GridSearchCV
import numpy as np

#create new KNN model
KNN2 = KNeighborsRegressor()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1,25)}

#use gridsearch to test all values for n_neighbors
KNN_gscv = GridSearchCV(KNN2, param_grid, cv=5)

#fit model to data
KNN_gscv.fit(exclude, y1)

#check top performing n_neighbors value
KNN_gscv.best_params_

{'n_neighbors': 6}

In [11]:
#check mean score for the top performing value of n_neighbors
KNN_gscv.best_score_

0.16634025873515443

check the first model with k = 5

In [12]:
from sklearn.neighbors import KNeighborsRegressor

KNN3 = KNeighborsRegressor(n_neighbors=6)
KNN3.fit(X_train, y_train1)
KNN3.score(X_test, y_test1)

0.2295557877773613

In [13]:
a = KNN3.predict(X_test)


In [14]:
#performance
performance = pd.DataFrame({'True Value':y_test1,
                            'Prediction': a,
                            'Error': y_test1-a})

performance

,True Value,Prediction,Error
0,0.759097,0.822521,-0.063424
1,0.662766,0.797297,-0.134532
2,0.811290,0.830685,-0.019395
3,0.879192,0.794306,0.084886
4,0.973441,0.861620,0.111821
...,...,...,...
95,0.962167,0.797060,0.165107
96,0.692127,0.799467,-0.107341
97,0.945043,0.871492,0.073552
98,0.762350,0.770986,-0.008637


## CD

In [15]:
X_train, X_test, y_train2, y_test2 = train_test_split(exclude, y2, test_size=0.2, random_state=1)

In [16]:
KNN4 = KNeighborsRegressor(n_neighbors=10)
KNN4.fit(X_train, y_train2)

KNeighborsRegressor(n_neighbors=10)

In [17]:
KNN4.score(X_test, y_test2)

0.19992336018897117

evaluating k

In [18]:
#create new KNN model
KNN5 = KNeighborsRegressor()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1,25)}

#use gridsearch to test all values for n_neighbors
KNN_gscv1 = GridSearchCV(KNN5, param_grid, cv=4)

#fit model to data
KNN_gscv1.fit(exclude, y2)

KNN_gscv1.best_params_

{'n_neighbors': 3}

In [19]:
KNN6 = KNeighborsRegressor(n_neighbors=3)
KNN6.fit(X_train, y_train2)
KNN6.score(X_test, y_test2)

0.2828397633756564

In [20]:
b = KNN6.predict(X_test)


In [21]:
performance1 = pd.DataFrame({'True Value':y_test2,
                             'Prediction': b,
                             'Error': y_test2-b})

performance1

,True Value,Prediction,Error
0,0.005532,0.009151,-0.003619
1,0.003641,0.014101,-0.010460
2,0.008803,0.011279,-0.002476
3,0.024755,0.017091,0.007664
4,0.023107,0.025950,-0.002843
...,...,...,...
95,0.031780,0.019558,0.012222
96,0.003432,0.011571,-0.008140
97,0.017330,0.016063,0.001267
98,0.007996,0.006980,0.001016


### Cm

In [22]:
X_train, X_test, y_train3, y_test3 = train_test_split(exclude, y3, test_size=0.2, random_state=1)


In [23]:
KNN7 = KNeighborsRegressor(n_neighbors=10)
KNN7.fit(X_train, y_train3)

KNeighborsRegressor(n_neighbors=10)

In [24]:
KNN7.score(X_test, y_test3)

0.19557574553951407

In [25]:
#create new KNN model
KNN8 = KNeighborsRegressor()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1,25)}

#use gridsearch to test all values for n_neighbors
KNN_gscv2 = GridSearchCV(KNN5, param_grid, cv=50)

#fit model to data
KNN_gscv2.fit(exclude, y3)

KNN_gscv2.best_params_

{'n_neighbors': 4}

In [26]:
KNN9 = KNeighborsRegressor(n_neighbors=4)
KNN9.fit(X_train, y_train3)
KNN9.score(X_test, y_test3)

0.2571201819863984

In [27]:
c = KNN9.predict(X_test)

In [28]:
performance2 = pd.DataFrame({'True Value':y_test3,
                             'Prediction': c,
                             'Error': y_test3-c})

performance2

,True Value,Prediction,Error
0,0.109919,0.124134,-0.014215
1,0.073966,0.121295,-0.047329
2,0.108302,0.130150,-0.021848
3,0.149516,0.120008,0.029508
4,0.173853,0.164440,0.009413
...,...,...,...
95,0.184703,0.145957,0.038745
96,0.087176,0.131481,-0.044305
97,0.171315,0.156029,0.015286
98,0.100759,0.100878,-0.000119


In [30]:
performance.to_excel(r'C:\Users\HP\Downloads\Polar_Graph_2.xlsx', index=False, engine='openpyxl', sheet_name='beta_te-CL')

In [31]:
performance1.to_excel(r'C:\Users\HP\OneDrive - Institut Teknologi Bandung\Sains Data Statistik\beta_te.xlsx', index=False, engine='openpyxl', sheet_name='beta_te-CD')

In [32]:
performance2.to_excel(r'C:\Users\HP\Downloads\Polar_Graph_1.xlsx', index=False, engine='openpyxl', sheet_name='beta_te-CM')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64783e9b-ffe1-4249-97cd-6bd7029f6d44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>